In [324]:
from proxy_chrome_browser import get_chromedriver, build_proxy_extension
import moviepy.editor as mpy
import glob
import os
from time import sleep
from tqdm import tqdm

In [325]:
br = get_chromedriver(headless=True,path='/home/miko/Downloads/chromedriver-v-106-0-5249-61')

In [326]:
def find_elm(tag,driver=br,_all=False,_by="css"):
    from selenium.webdriver.common.by import By
    if _by=="css":
        _by=By.CSS_SELECTOR
    elif _by=="link_text":
        _by=By.PARTIAL_LINK_TEXT
    elif _by=="link_text_exact":
        _by=By.LINK_TEXT
        
    try:
        if _all:
            out = driver.find_elements(by=_by,value=tag)
        else:
            out = driver.find_element(by=_by,value=tag)
    except:
        #print('None')
        out = None
    return out

In [327]:
def save_screenshot(elm,filename,scrollY=None):
    assert filename[-3:] == "png", "filename should end with png"
    scroll_into_view(elm,scrollY)
    with open(filename,"wb") as f:
        f.write(elm.screenshot_as_png)

In [328]:
def scroll_into_view(elm,scrollY=None):
     #source: -https://stackoverflow.com/questions/25733877/selenium-how-do-you-check-scroll-position
     #loc = elm.location
     #current_view_script = """ var doc = document.documentElement;
     #var left = (window.pageXOffset || doc.scrollLeft) - (doc.clientLeft || 0);
     #var top = (window.pageYOffset || doc.scrollTop)  - (doc.clientTop || 0);
        
     #taking screenshot will scroll into view. IF this stops working can use javascript above
     elm.screenshot_as_png
     sleep(.2)
     # scroll a bit to add margin (prevent floating login bar from obscuring elm);
     if scrollY:
         br.execute_script(f"window.scrollBy(0,{scrollY})", "")
         sleep(.5)


In [329]:
#set to dark theme
br.get("https://www.reddit.com")

In [330]:

icon = find_elm("button[id='USER_DROPDOWN_ID']")
icon.click()
sleep(.5)
dark_btn = find_elm("button[data-redditstyle='true']")
dark_btn.click()
sleep(.5)
# if for some reason click isnt working. set it manually
#input("Confirm darkmode has been set (press enter)")

In [360]:
#post_url = "https://www.reddit.com/r/AskReddit/comments/ypzsqh/men_of_reddit_what_are_some_less_obvious_red/"
#post_url = "https://www.reddit.com/r/AskReddit/comments/ycb6hp/whats_a_harsh_reality_that_everybody_needs_to_hear/"
#post_url = "https://www.reddit.com/r/AskReddit/comments/y5bcsl/nonamericans_what_do_you_think_every_american/"
#post_url ="https://www.reddit.com/r/AskReddit/comments/ydz1j5/what_is_25_years_too_old_for/"
post_url = "https://www.reddit.com/r/AskReddit/comments/y5wxnn/what_is_the_stupidest_thing_a_large_amount_of/?sort=confidence"

In [361]:
br.get(post_url)

In [362]:
#br.switch_to.active_element.text

In [363]:
sleep(10)

In [364]:
# expand replies
num_replies_threshold = 90
get_replies = True
num_total_loops = 5
current_loop_num = 0
clicked_links = []
while get_replies:
    more_replies_divs = find_elm("div[id^='moreComments']",_all=True)
    expandable_divs = []
    for div in more_replies_divs:
        more_replies_link = find_elm("p",driver=div)
        #p = br.execute_script("return arguments[0].parentNode;",more_replies_link)
        if not more_replies_link:
            continue
        if more_replies_link.text == '' or 'loading..' in more_replies_link.text:
            continue
        num_replies = int(more_replies_link.text.split()[0].replace(",",""))
        if num_replies >= num_replies_threshold:
            if more_replies_link not in clicked_links:
                expandable_divs.append(more_replies_link)
            else:
                continue
    for div in expandable_divs:
        #link_elm = find_elm("div[data-testid^='search-scope-pill-text']")
        try:
            scroll_into_view(more_replies_link)
            div.click()
            sleep(1)
        except:
            continue  
    if len(expandable_divs) == 0:
        get_replies = False
    if current_loop_num >= num_total_loops:
        get_replies = False
    else:
        current_loop_num += 1
    print("getting more replies")
    #todo: find a better stopping condition

getting more replies
getting more replies
getting more replies
getting more replies
getting more replies
getting more replies


In [365]:
# get comments
comments = find_elm("div[class*='Comment']",_all=True)

In [366]:
#prepend original post to comments
post = [find_elm("div[data-test-id='post-content']"),]
post.extend(comments)
comments = post

In [367]:
len(comments)

394

In [368]:
submission_title = post_url.split("/")[-2]
#asset_dir="assets/demo_png_2_vid"
asset_dir="/mnt/n/data/reddit-yt-tts/queued"
completed_dir="/mnt/n/data/reddit-yt-tts/completed"
#thread_ids = []

In [369]:
def save_comment_text(filename,comment,selector='p'):
    comment_text = " ".join([x.text for x in find_elm(selector,driver=comment,_all=True)])
    with open(filename,"w") as f:
        f.write(comment_text)

In [370]:
#save threaded comment screenshot as png to directory
vote_count_threshold = 450
thread_count = 0
parent_comment_stack = [dict(comment=comments[0],saved=True)]
prev_comment_lvl = 1
prev_comment = comments[0]
prev_comment_id = -1
i = 0
for comment in tqdm(comments):
    # filter deleted comments
    if "Comment deleted by user" in comment.text:
        continue
    
    # some comments arnt expanded, skipping
    if len(comment.text.split("\n")) <= 4:
           continue
           
    #filter highly voted comments
    vote_div = find_elm("div[id^='vote-arrows']",driver=comment)
    if not vote_div or vote_div.text == '':
        continue
        
    multiply_1k = 'k' in vote_div.text
    if multiply_1k:
        vote_count = float(vote_div.text.replace("k","")) * 1000
    else:
        vote_count = int(vote_div.text)
    
    if i == 0:
        comment_lvl = 1
    else:
        comment_lvl = find_elm("span",comment).text
        comment_lvl = int(comment_lvl.lower().replace("level","").strip())

    #if "1" in comment_lvl:
    if comment_lvl == 1:
        thread_count += 1
        user = comment.text.split("\n")[1]
        vote_count_txt = vote_div.text.replace(".","_")
        thread_id = f"{thread_count:04}-{user}-{vote_count_txt}"
        
        comment_id=0
    else:
        comment_id+=1
        
    # Save parent comments even if threshold is not met
    if comment_lvl > prev_comment_lvl:
        # entering a nested thread, mark new parent...
        parent_comment_stack.append(dict(comment=prev_comment,comment_id=prev_comment_id,saved=False))
    elif comment_lvl < prev_comment_lvl:
        # leaving a nested thread, remove previous parent
        parent_comment_stack = parent_comment_stack[:-1]
        
    
    # Check whether comment should be saved
    save_comment = False
    if vote_count >= vote_count_threshold:
        save_comment = True
    
    #save parent comment
    parent_comment = parent_comment_stack[-1]
    if save_comment and not parent_comment["saved"]:
        #todo refactor to fn
        folder = f"{asset_dir}/{submission_title}/{thread_id}"
        if not os.path.exists(folder):
            os.makedirs(folder)
        save_screenshot(parent_comment['comment'],f"{folder}/{parent_comment['comment_id']:04}.png")
        save_comment_text(filename=f"{folder}/{parent_comment['comment_id']:04}.txt",comment=parent_comment['comment'])
        parent_comment["saved"] = True
        
    #Save comment
    if save_comment:
        #thread_ids.append(thread_id)
        folder = f"{asset_dir}/{submission_title}/{thread_id}"
        if not os.path.exists(folder):
            os.makedirs(folder)
        save_screenshot(comment,f"{folder}/{comment_id:04}.png")
        if i == 0:
            save_screenshot(comment,f"{folder}/{comment_id:04}.png",scrollY=-55)
            save_comment_text(filename=f"{folder}/{comment_id:04}.txt",comment=comment,selector='h1')
        else:
            save_screenshot(comment,f"{folder}/{comment_id:04}.png")
            save_comment_text(filename=f"{folder}/{comment_id:04}.txt",comment=comment)
    
    # init vars for next iteration
    prev_comment_lvl = comment_lvl
    prev_comment = comment
    prev_comment_id = comment_id
    i+=1

100%|█████████████████████████████████████████| 394/394 [03:48<00:00,  1.72it/s]


In [371]:
#save_screenshot(comments[1],f"{asset_dir}/test.png")

In [372]:
thread_ids = [x.split("/")[-1] for x in glob.glob(f"{asset_dir}/{submission_title}/*")]

In [343]:
def build_thread_vid(thread_id):
    images = glob.glob(f"{asset_dir}/{submission_title}/{thread_id}/*.png")
    images = sorted(images)
    audio_file_dir = f"{completed_dir}/{submission_title}/{thread_id}"
    # https://zulko.github.io/moviepy/examples/examples.html
    clips = []
    audio_clips = []
    total_duration = 0
    transition_duration = 0
    for i,image in enumerate(images):
      
        image_file = image.split("/")[-1]
        audio_file = image_file.replace(".png",".wav")
        try:
            audio_clip = mpy.AudioFileClip(f"{audio_file_dir}/{audio_file}").set_start(total_duration)
        except OSError as e:
            print(f"audio file {audio_file_dir}/{audio_file} not found")
            continue
        audio_clips.append(audio_clip)
        audio_duration = audio_clip.duration
        
        clip = mpy.ImageClip(image).set_duration(audio_duration).set_start(total_duration).set_position(("center", "center"))
        bg_clip = mpy.ImageClip("assets/Qv79akqGQt0.png").set_duration(audio_duration).set_start(total_duration)
        
        #clip = clip.set_start(5) # TODO: change duration based on audio
        #clip = clip.resize((1080,1920)) #TODO add tts audio
        clips.append(bg_clip)
        clips.append(clip)
        
        total_duration += audio_duration + transition_duration
  
    if len(clips) > 0 :
        final_audio_clip = mpy.CompositeAudioClip(audio_clips)
        final_clip = mpy.CompositeVideoClip(clips)
        final_clip.audio = final_audio_clip
        final_clip.write_videofile(f"{asset_dir}/{submission_title}/{thread_id}/thread_clip.mp4",fps=30,logger=None)
    

In [344]:
thread_ids = sorted(list(set(thread_ids)))

In [345]:
f"{asset_dir}/{submission_title}/{thread_ids[0]}"

'/mnt/n/data/reddit-yt-tts/queued/what_is_25_years_too_old_for/0001-Posted by-38_2k'

In [70]:
# Execute TTS at this point (on windows GPU) then run below
input("Execute TTS at this point (on windows GPU) then run below...(press enter)")

Execute TTS at this point (on windows GPU) then run below...(press enter) 


''

In [346]:
# save threads to clips
# vote_div.text
thread_ids = sorted(list(set(thread_ids)))
for thread in tqdm(thread_ids):
    print(thread)
    build_thread_vid(thread)

  0%|                                                    | 0/43 [00:00<?, ?it/s]

0001-Posted by-38_2k


  2%|█                                           | 1/43 [00:02<01:31,  2.19s/it]

0002-BenZonne-11_5k


  5%|██                                          | 2/43 [00:15<05:53,  8.63s/it]

...

audio file /mnt/n/data/reddit-yt-tts/completed/what_is_25_years_too_old_for/0052-Complex-Whereas-5787-1_4k/0000.wav not found
0053-vulpesvulpex-683
audio file /mnt/n/data/reddit-yt-tts/completed/what_is_25_years_too_old_for/0053-vulpesvulpex-683/0000.wav not found
0059-HocusSnood-1_0k


 98%|██████████████████████████████████████████ | 42/43 [09:20<00:01,  1.17s/it]

audio file /mnt/n/data/reddit-yt-tts/completed/what_is_25_years_too_old_for/0059-HocusSnood-1_0k/0000.wav not found
audio file /mnt/n/data/reddit-yt-tts/completed/what_is_25_years_too_old_for/0059-HocusSnood-1_0k/0001.wav not found
0069-EJ6EM1-464


100%|███████████████████████████████████████████| 43/43 [09:21<00:00, 13.05s/it]

audio file /mnt/n/data/reddit-yt-tts/completed/what_is_25_years_too_old_for/0069-EJ6EM1-464/0000.wav not found


In [347]:
# save c to 

In [348]:
videos = glob.glob(f"{asset_dir}/{submission_title}/*/*.mp4")
videos = sorted(list(set(videos)))

In [349]:
all_video_clips = [mpy.VideoFileClip(video) for video in videos]

In [350]:
videos_w_sound_corrected = []
for clip in all_video_clips:
    clip.audio = clip.audio.fx(mpy.afx.audio_normalize)
    videos_w_sound_corrected.append(clip)

In [351]:
transition = mpy.VideoFileClip("assets/tvstatic-180-720.mp4")

In [352]:
transition2 = mpy.VideoFileClip("assets/free-vhs-overlay-1080-720.mp4")

In [353]:
transition2.audio = transition.audio
transition2 = transition2.set_duration(.7)
#transition = transition.resize(1080,720) resizing crashes kernal reesizing 
#using ffmpeg on cli instead:  https://trac.ffmpeg.org/wiki/Scaling

#split bg music into parts: https://unix.stackexchange.com/questions/280767/how-do-i-split-an-audio-file-into-multiple
#ffmpeg -i somefile.mp3 -f segment -segment_time 3 -c copy out%03d.mp3


In [354]:
final_video = mpy.concatenate_videoclips(videos_w_sound_corrected,method="compose",transition=transition2)

In [373]:
glob.glob("assets/bg_music/*.opus")

['assets/bg_music/best-of-lofi-2021-005.opus',
 'assets/bg_music/best-of-lofi-2021-003.opus',
 'assets/bg_music/best-of-lofi-2021-013.opus',
 'assets/bg_music/best-of-lofi-2021-008.opus',
 'assets/bg_music/best-of-lofi-2021-000.opus',
 'assets/bg_music/best-of-lofi-2021-007.opus',
 'assets/bg_music/best-of-lofi-2021-002.opus',
 'assets/bg_music/Sátyr x Phlocalyst - Aiode 🌘 [lofi hip hop_relaxing beats]-n5nscOfa18w.opus',
 'assets/bg_music/best-of-lofi-2021-001.opus',
 'assets/bg_music/best-of-lofi-2021-011.opus',
 'assets/bg_music/best-of-lofi-2021-006.opus',
 'assets/bg_music/best-of-lofi-2021-010.opus',
 'assets/bg_music/best-of-lofi-2021-012.opus',
 'assets/bg_music/Best of lofi hip hop 2021 ✨ - beats to relax_study to-n61ULEU7CO0.opus',
 'assets/bg_music/best-of-lofi-2021-004.opus',
 'assets/bg_music/best-of-lofi-2021-009.opus']

In [375]:


def get_audio(v_duration):
    import random
    get_audio = True
    sel_file_clips = []
    sel_file_names = []
    while get_audio:
        
        audio_files = glob.glob("assets/bg_music/*.opus")
        
        sel_file = random.choice(audio_files)
        
        sel_file_clips.append(mpy.AudioFileClip(sel_file))
        sel_file_names.append(sel_file)
        
        all_duration = [ x.duration for x in sel_file_clips]
        
        total_duration = 0
        
        for duration in all_duration:
            total_duration += duration
        
        if total_duration >= v_duration:
            get_audio = False
    
    out_audio = mpy.concatenate_audioclips(sel_file_clips).set_duration(v_duration)
    return out_audio, sel_file_names 
        

In [376]:
#import moviepy.video.fx as vfx
#import moviepy.audio.fx as afx
bg_music,bg_music_files = get_audio(final_video.duration)

bg_music = bg_music.fx( mpy.afx.audio_normalize).fx( mpy.afx.volumex, 0.3).fx( mpy.afx.audio_fadein, 1.0)
tts_audio = final_video.audio

In [377]:
bgm_w_tts_audio = mpy.CompositeAudioClip([bg_music,tts_audio])

In [378]:
final_video.audio = bgm_w_tts_audio

In [358]:
final_video.write_videofile(f"assets/final_videos/{submission_title}.mp4",fps=30,logger=None)

In [406]:
# Save video credits
def check_wildcard(file,credit):
    prefix = credit["filename"].split(".*")[0]
    return prefix in file
        
def get_audio_credits(audio_files):
    import yaml
    with open("assets/bg_music/asset_credits.yml","r") as f:
        credits = yaml.load(f,yaml.FullLoader)
    active_credits = []
    for credit in credits["assets"]:
        for file in audio_files:
            if ".*" in credit["filename"] and check_wildcard(file,credit):
                active_credits.append(credit)
            elif credit["filename"] == file:
                active_credits.append(credit)
    return active_credits

In [408]:
credits = get_audio_credits(bg_music_files)
all_credit_texts = []
for credit in credits:
    credit_text = f"""
    - {credit['artist']} - {credit['title']} 
    - Provided by Lofi Girl
    - Watch: {credit['link']}
    - Listen: {credit['spotify']}
    """
    all_credit_texts.append(credit_text)
audio_credits = "--------\n".join(set(all_credit_texts))
#print(audio_credits)

In [410]:
with open(f"assets/final_video_credits/{submission_title}.txt","w") as f:
    f.write(audio_credits)

In [411]:
1+1

2

In [383]:
!pipenv install pyyaml

Installing pyyaml...
Adding pyyaml to Pipfile's [packages]...
✔ Installation Succeeded 
Pipfile.lock (84cd22) out of date, updating to (20dd62)...
Locking [dev-packages] dependencies...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Updated Pipfile.lock (20dd62)!
Installing dependencies from Pipfile.lock (20dd62)...
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 5/5 — 00:00:038;5;1m▉ 4/5 — 00:00:00m▉ 3/5 — 00:00:008;5;1m▉ 2/5 — 00:00:00m▉ 1/5 — 00:00:00m▉ 0/5 — 00:00:00


In [87]:
#create transition clips
!ls assets/

 bg-img.jpg
 bg-img.png
 bg-test.mp3
 demo_png_2_vid
 demos
 demo_trts
'Free Glitch Transition Overlay Pack №2 HD 4K-7HP3IWdCc3s.mkv'
'Free VHS overlay effect-KQwJ5AM8zns.f137.mp4.part'
 pngtree-soft-small-bokeh-transparency-background-png-image_2326962.jpg
 reddit_whats_your_most_im_with_the_boomers_on.mp4
 test_final_vid-fast.mp4
 test_final_vid.mp4
'TV static glitch effect                                                                (for editing)-cYgODiuD5co.mp4'
'WYS – Snowman-0WmRB9Iz-30.opus'
